In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from keras.models import Sequential
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Dense
from keras import layers
from keras.models import Sequential
from keras import regularizers
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
# setting earlystopping
early = EarlyStopping(monitor = 'val_loss',
              mode = 'min',
              patience = 3, 
              restore_best_weights = True)

# setting modelcheckpoint
check = ModelCheckpoint('./m_checkpoint',
               monitor = 'val_loss',
               save_best_only = True,
               mode = 'min')

# creating list for callbacks
call_list = [early, check]

In [3]:
# loading data from directory
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range = 20,
        validation_split = .3,
        horizontal_flip=True)

train_generator = datagen.flow_from_directory(
        '/kaggle/input/granite_slabs/granite_slabs',
        subset = 'training',
        target_size=(224, 224),
        batch_size=32,
        color_mode = 'rgb',
        class_mode= 'categorical')

validation_generator = datagen.flow_from_directory(
        '/kaggle/input/granite_slabs/granite_slabs',
        subset = 'validation',
        target_size=(224, 224),
        batch_size=32,
        class_mode= 'categorical')

Found 452 images belonging to 6 classes.
Found 191 images belonging to 6 classes.


In [4]:
# Instantiating model
model = Sequential()

# input Convolutional layer
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))



# # Adding 1st conv layer
model.add(Conv2D(128,
                kernel_size = 3,
                activation = 'relu',
                kernel_regularizer = regularizers.l2(.01)))

# # adding pooling
# model.add(MaxPooling2D(pool_size = 3))

# # adding dropout
# # model.add(Dropout(rate = .5))

# # Adding 2nd conv layer
# model.add(Conv2D(128,
#                  kernel_size = 3,
#                  activation = 'relu',
#                  kernel_regularizer = regularizers.l2(.01)))

# # Adding 2nd max pooling
# model.add(MaxPooling2D(pool_size= 2))

# # adding dropout
# # model.add(Dropout(rate = .25))
          
# # Adding 3rd conv layer
# model.add(Conv2D(256,
#                  kernel_size=3,
#                  activation='relu',
#                  kernel_regularizer = regularizers.l2(.01)))

# # Adding 3rd max pooling
# model.add(MaxPooling2D(pool_size= 2))

# # Adding 4th conv layer
# model.add(Conv2D(256,
#                  kernel_size=3,
#                  activation='relu', 
#                  kernel_regularizer = regularizers.l2(.01)))

# # Adding 4th max pooling
# model.add(MaxPooling2D(pool_size= 2))

# # Adding 5th conv layer
# model.add(Conv2D(256,
#                  kernel_size=1,
#                  activation='relu',
#                  kernel_regularizer = regularizers.l2(.01)))

# # Adding 5th max pooling
# model.add(MaxPooling2D(pool_size= 2))

# # Adding 6th conv layer
# model.add(Conv2D(256,
#                  kernel_size=1,
#                  activation='relu',
#                  kernel_regularizer = regularizers.l2(.01)))

# Adding 6th max pooling
model.add(MaxPooling2D(pool_size= 4))

# # Adding 7th conv layer
# model.add(Conv2D(256,
#                  kernel_size=1,
#                  activation='relu',
#                  kernel_regularizer = regularizers.l2(.01)))

# # Adding 7th max pooling
# model.add(MaxPooling2D(pool_size= 2))

model.add(layers.Flatten())

# model.add(Dense(256,
#                activation = 'relu'))
# model.add(Dense(128,
#                activation = 'relu'))
# model.add(Dense(64,
#                activation = 'relu'))
model.add(Dense(128,
               activation = 'relu',
               kernel_regularizer = regularizers.l2(.01)))
model.add(layers.Dense(6, activation = 'softmax',
                      kernel_regularizer = regularizers.l2(.01)))

In [5]:
# compiling model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],)

In [6]:
# checking summary of model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 55, 55, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 387200)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               49561728  
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 774       
Total params: 49,638,150
Trainable params: 49,638,150
Non-trainable params: 0
________________________________________________________________

In [7]:
# fitting model
history = model.fit_generator(
        train_generator,
        steps_per_epoch=50,
        epochs=12,
        validation_data=validation_generator,
        validation_steps=12,
        workers = -1,
        verbose = 1,
        callbacks = call_list)

Epoch 1/12
50/50 [==============================] - 392s 8s/step - loss: 4.2533 - acc: 0.2650 - val_loss: 2.1714 - val_acc: 0.2644
Epoch 2/12
50/50 [==============================] - 381s 8s/step - loss: 1.8452 - acc: 0.4819 - val_loss: 1.8528 - val_acc: 0.4293
Epoch 3/12
50/50 [==============================] - 371s 7s/step - loss: 1.6095 - acc: 0.5454 - val_loss: 1.8897 - val_acc: 0.3796
Epoch 4/12
50/50 [==============================] - 376s 8s/step - loss: 1.4886 - acc: 0.5763 - val_loss: 1.7178 - val_acc: 0.4555
Epoch 5/12
50/50 [==============================] - 374s 7s/step - loss: 1.4668 - acc: 0.5863 - val_loss: 2.0839 - val_acc: 0.3429
Epoch 6/12
50/50 [==============================] - 372s 7s/step - loss: 1.4346 - acc: 0.5556 - val_loss: 1.7194 - val_acc: 0.4162
Epoch 7/12
50/50 [==============================] - 377s 8s/step - loss: 1.3568 - acc: 0.6175 - val_loss: 1.7980 - val_acc: 0.4607


In [8]:
feat_extractor = Model(model.input, outputs=model.get_layer("dense_2").output)

In [9]:
feat_extractor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 222, 222, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 55, 55, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 387200)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               49561728  
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 774       
Total para

In [10]:
# saving model
# model.save('./model')

In [11]:
# evaluating model performance
model.evaluate_generator(train_generator, steps = 5, workers = -1, verbose = 1)

5/5 [==============================] - 31s 6s/step


[1.4364753246307373, 0.58125]